In [2]:
import pandas as pd
import numpy as np

In [28]:
ca_df = pd.read_csv("./data/registration/ca_ev_registrations_public.csv", dtype={'County GEOID': str})
ca_df = ca_df.drop(columns=['Vehicle ID', 'DMV ID', 'DMV Snapshot', 'Registration Expiration Date', 'Geography'])
ca_df

,County GEOID,Registration Valid Date,State Abbreviation,Vehicle Name
0,06099,2011-01-01,CA,Chevrolet Volt
1,06105,2011-01-01,CA,Nissan Leaf
2,06103,2011-01-01,CA,Chevrolet Volt
3,06099,2011-01-01,CA,Tesla Roadster
4,06099,2011-01-01,CA,Tesla Roadster
...,...,...,...,...
2542438,06075,2020-01-01,CA,Toyota Prius Plug-in
2542439,06075,2020-01-01,CA,Toyota Prius Plug-in
2542440,06075,2020-01-01,CA,Toyota Prius Plug-in
2542441,06075,2020-01-01,CA,Toyota Prius Plug-in


In [31]:
ca_df['Drivetrain Type'] = ca_df['Vehicle Name'].apply(lambda x: 'PHEV' if (('hybrid' in x.lower()) or ('plug' in x.lower())) else 'BEV')
ca_df['Vehicle Make'] = ca_df['Vehicle Name'].apply(lambda x: x.split()[0])
ca_df['Vehicle Model'] = ca_df['Vehicle Name'].apply(lambda x: ' '.join(x.split()[1:]))
ca_df = ca_df.rename(columns={'Registration Valid Date': 'Registration Date', 'State Abbreviation': 'State'})
ca_df ca_df.drop(columns=['Vehicle Name'])

,County GEOID,Registration Date,State,Drivetrain Type,Vehicle Make,Vehicle Model
0,06099,2011-01-01,CA,BEV,Chevrolet,Volt
1,06105,2011-01-01,CA,BEV,Nissan,Leaf
2,06103,2011-01-01,CA,BEV,Chevrolet,Volt
3,06099,2011-01-01,CA,BEV,Tesla,Roadster
4,06099,2011-01-01,CA,BEV,Tesla,Roadster
...,...,...,...,...,...,...
2542438,06075,2020-01-01,CA,PHEV,Toyota,Prius Plug-in
2542439,06075,2020-01-01,CA,PHEV,Toyota,Prius Plug-in
2542440,06075,2020-01-01,CA,PHEV,Toyota,Prius Plug-in
2542441,06075,2020-01-01,CA,PHEV,Toyota,Prius Plug-in


In [32]:
ca_df

,County GEOID,Registration Date,State,Vehicle Name,Drivetrain Type,Vehicle Make,Vehicle Model
0,06099,2011-01-01,CA,Chevrolet Volt,BEV,Chevrolet,Volt
1,06105,2011-01-01,CA,Nissan Leaf,BEV,Nissan,Leaf
2,06103,2011-01-01,CA,Chevrolet Volt,BEV,Chevrolet,Volt
3,06099,2011-01-01,CA,Tesla Roadster,BEV,Tesla,Roadster
4,06099,2011-01-01,CA,Tesla Roadster,BEV,Tesla,Roadster
...,...,...,...,...,...,...,...
2542438,06075,2020-01-01,CA,Toyota Prius Plug-in,PHEV,Toyota,Prius Plug-in
2542439,06075,2020-01-01,CA,Toyota Prius Plug-in,PHEV,Toyota,Prius Plug-in
2542440,06075,2020-01-01,CA,Toyota Prius Plug-in,PHEV,Toyota,Prius Plug-in
2542441,06075,2020-01-01,CA,Toyota Prius Plug-in,PHEV,Toyota,Prius Plug-in
